# GNN モジュールの作成

GNNモデルを作成する際、既存のGNNモジュールを単純に積み重ねるだけでは不十分な場合がある。
例えば、ノードの重要性やエッジの重みを考慮して隣接情報を集約する新しい方法を考案したいときがある。

このチュートリアルでは、以下のことを学ぶ。

- DGLのメッセージパッシングAPIを理解する。
- 自分でGraphSAGE畳み込みモジュールを実装する。


このチュートリアルは、すでに :doc:`<1_introduction>`を理解していることを前提としています。

In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

## メッセージパッシングとGNN

DGLは、[Gilmer et al.](https://arxiv.org/abs/1704.01212)によって提案されたメッセージパッシングニューラルネットワークに触発された*メッセージパッシングパラダイム*に従う。基本的に、彼らは多くのGNNモデルが以下のフレームワークに適合することを示した。

\begin{align}m_{u\to v}^{(l)} = M^{(l)}\left(h_v^{(l-1)}, h_u^{(l-1)}, e_{u\to v}^{(l-1)}\right)\end{align}

\begin{align}m_{v}^{(l)} = \sum_{u\in\mathcal{N}(v)}m_{u\to v}^{(l)}\end{align}

\begin{align}h_v^{(l)} = U^{(l)}\left(h_v^{(l-1)}, m_v^{(l)}\right)\end{align}


ここで、$M^{(l)}$を*メッセージ関数*、$\sum$を*集約関数*、$U^{(l)}$を*更新関数*と呼ぶ。ここでの$\sum$は任意の関数を表すことができ、必ずしも総和である必要はない。



例えば、[GraphSAGE畳み込み (Hamilton et al., 2017)](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf)は以下の数式で表される。

\begin{align}h_{\mathcal{N}(v)}^k\leftarrow \text{Average}\{h_u^{k-1},\forall u\in\mathcal{N}(v)\}\end{align}

\begin{align}h_v^k\leftarrow \text{ReLU}\left(W^k\cdot \text{CONCAT}(h_v^{k-1}, h_{\mathcal{N}(v)}^k) \right)\end{align}

ここでのメッセージパッシングには、方向性がある。  
一般にノード$u$からノード$v$に送信されるメッセージは、逆方向のノード$v$からノード$u$に送信されるメッセージとは異なる場合がある。

DGLは、:class:`dgl.nn.SAGEConv <dgl.nn.pytorch.SAGEConv>`を介してGraphSAGEをサポートしているが、  
以下に、自分でGraphSAGE畳み込みを実装する方法を示す。

In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata["h"] = h
            # update_all でメッセージの集約を行う
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

このコードの中心部分は、:func:`g.update_all <dgl.DGLGraph.update_all>`関数である。
この関数は、隣接ノードの特徴を集めて平均化する。`update_all`関数には3つの概念がある。

* メッセージ関数``fn.copy_u('h', 'm)``は、ノード特徴を名前``'h'``で*メッセージ*としてコピーし、  
  隣接ノードに名前``'m'``で送信する。

* 集約関数は、```fn.mean('m', 'h_N')``` で、名前``'m'``で受信したすべてのメッセージを平均化し、  
  新しいノード特徴``'h_N'``として保存する。

* ``update_all`` は、DGLにすべてのノードとエッジに対してメッセージと集約関数を呼び出すように指示する。

このコードの中心部分は、:func:`g.update_all <dgl.DGLGraph.update_all>`関数である。  
:func:`g.update_all <dgl.DGLGraph.update_all>`関数は、隣接ノードの特徴を集めて平均化する。

その後、自分のGraphSAGE畳み込み層を積み重ねて、多層のGraphSAGEネットワークを作成できる。

In [3]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

### 学習ループ   
以下のデータロードと学習ループのコードは、`1_introduction.ipynb` からコピーされています。  

In [5]:
g.ndata["train_mask"]

tensor([ True,  True,  True,  ..., False, False, False])

In [6]:
g.ndata["val_mask"]

tensor([False, False, False,  ..., False, False, False])

In [7]:
g.ndata["test_mask"]

tensor([False, False, False,  ...,  True,  True,  True])

In [4]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.950, val acc: 0.058 (best 0.058), test acc: 0.064 (best 0.064)
In epoch 5, loss: 1.878, val acc: 0.342 (best 0.342), test acc: 0.340 (best 0.340)
In epoch 10, loss: 1.745, val acc: 0.450 (best 0.450), test acc: 0.418 (best 0.418)
In epoch 15, loss: 1.547, val acc: 0.494 (best 0.494), test acc: 0.498 (best 0.489)
In epoch 20, loss: 1.289, val acc: 0.560 (best 0.560), test acc: 0.545 (best 0.545)
In epoch 25, loss: 0.995, val acc: 0.618 (best 0.618), test acc: 0.594 (best 0.594)
In epoch 30, loss: 0.705, val acc: 0.674 (best 0.674), test acc: 0.653 (best 0.653)
In epoch 35, loss: 0.461, val acc: 0.722 (best 0.722), test acc: 0.696 (best 0.696)
In epoch 40, loss: 0.286, val acc: 0.748 (best 0.748), test acc: 0.720 (best 0.720)
In epoch 45, loss: 0.175, val acc: 0.752 (best 0.752), test acc:

## さらにカスタマイズする

DGLでは、``dgl.function``パッケージの下に多くの組み込みのメッセージと集約関数が用意されている。
詳細は、`APIドキュメント <apifunction>`を参照してください。

これらのAPIを使用すると、新しいグラフ畳み込みモジュールを迅速に実装できる。  
例えば、以下は、重み付き平均を使用して隣接ノードの表現を集約する新しい ``SAGEConv`` を実装している。
``edata``はエッジ特徴を保持でき、メッセージパッシングにも使用できる。

In [8]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata["h"] = h
            g.edata["w"] = w
            g.update_all(
                message_func=fn.u_mul_e("h", "w", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

このデータセットのグラフはエッジの重みを持っていないため、モデルの ``forward()`` 関数ですべてのエッジの重みを1に手動で割り当てている。
edge weigthを考慮したい場合は、自分でエッジの重みを設定してください。

In [9]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device)) # torch.ones(g.num_edges(), 1).to(g.device) はエッジの重みが1の場合
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.951, val acc: 0.162 (best 0.162), test acc: 0.149 (best 0.149)
In epoch 5, loss: 1.887, val acc: 0.330 (best 0.330), test acc: 0.346 (best 0.346)
In epoch 10, loss: 1.764, val acc: 0.576 (best 0.576), test acc: 0.575 (best 0.575)
In epoch 15, loss: 1.576, val acc: 0.564 (best 0.576), test acc: 0.559 (best 0.575)
In epoch 20, loss: 1.330, val acc: 0.604 (best 0.604), test acc: 0.612 (best 0.612)
In epoch 25, loss: 1.048, val acc: 0.654 (best 0.654), test acc: 0.663 (best 0.663)
In epoch 30, loss: 0.767, val acc: 0.702 (best 0.702), test acc: 0.717 (best 0.717)
In epoch 35, loss: 0.523, val acc: 0.740 (best 0.740), test acc: 0.751 (best 0.751)
In epoch 40, loss: 0.336, val acc: 0.758 (best 0.758), test acc: 0.774 (best 0.774)
In epoch 45, loss: 0.211, val acc: 0.766 (best 0.766), test acc: 0.777 (best 0.777)
In epoch 50, loss: 0.132, val acc: 0.768 (best 0.768), test acc: 0.779 (best 0.774)
In epoch 55, loss: 0.085, val acc: 0.766 (best 0.768), test acc: 0.781 (best 0

## ユーザが定義した関数によるさらなるカスタマイズ
DGLは、ユーザが定義したメッセージ関数と集約関数を使用して、最大限のの表現を可能にしている。  
以下は、``fn.u_mul_e('h', 'w', 'm')`` と同等のユーザが定義したメッセージ関数である。

In [8]:
def u_mul_e_udf(edges):
    return {"m": edges.src["h"] * edges.data["w"]}

``edges``は、すべてのエッジに対して、ソースノード特徴、エッジ特徴、およぼターゲットノード特徴を表す ``src``、``data``、``dst`` の3つのメンバーを持っている。

集約関数も自分で定義可能である。例えば、以下は受信したメッセージを平均化する組み込みの ``fn.mean('m', 'h_N')`` 関数と同等である。

In [9]:
def mean_udf(nodes):
    return {"h_N": nodes.mailbox["m"].mean(1)}

要するに、DGLはノードを入次数でグループ化し、各グループに対してDGLは受信したメッセージを第2次元に沿ってスタックする。その後、第2次元に沿って平均化を行い、メッセージを集約する。

例. 入次数 3 (40, 1) -> (40, 3) -> (40, 1)

ユーザが定義した関数でメッセージと集約関数をカスタマイズする詳細については、`APIリファレンス <apiudf>`を参照してください。

## カスタムGNNモジュールのベストプラクティス

DGL は、以下の順序でカスタムGNNモジュールを作成することを推奨しています。


- ``dgl.nn`` モジュールを使用する。
- ``dgl.nn.functional`` 関数を使用する。これには、ノードごとに入力エッジに対してソフトマックスを計算するなど、より低レベルの複雑な操作が含まれている。
- 組み込みのメッセージと集約関数を使用して ``update_all`` を使用する。
- ユーザが定義したメッセージまたは集約関数を使用する。 

## What’s next?

-  `Writing Efficient Message Passing
   Code <guide-message-passing-efficient>`.


In [10]:
# Thumbnail credits: Representation Learning on Networks, Jure Leskovec, WWW 2018
# sphinx_gallery_thumbnail_path = '_static/blitz_3_message_passing.png'